<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/Proyecto_TFM/Models/MotileDetection/PostBestModel/ExtractingROIFromVideoList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creando procesamiento de Frames para extracción de secciones de espermatozoides móviles en un conjunto de videos.

In [1]:
%%capture
!pip install ultralytics

In [2]:
%%capture
!wget https://www.dropbox.com/scl/fi/maqbuw2e0s7ywa63dofun/bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt&st=04t3nonx&dl=0
!mv bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt bestSmall1920.pt

In [5]:
%%capture
# Descargar el archivo ZIP desde la URL
!wget https://www.dropbox.com/scl/fi/vnyl0lbmzarkfm7jrt88h/Videos.zip?rlkey=4i3y4so9xnomins88tpgdls0z&st=mgjlmjb3&dl=0

# Crear la carpeta "Videos" si no existe
!mkdir -p Videos

!mv Videos.zip?rlkey=4i3y4so9xnomins88tpgdls0z Videos.zip

# Descomprimir el archivo ZIP en la carpeta "Videos"
!unzip -o Videos.zip -d Videos


In [4]:
from ultralytics import YOLO
# Load a pretrained YOLO model (recommended for training)
model = YOLO('bestSmall1920.pt')

In [6]:
import cv2
import os

def extraer_primer_frame_video(video_path):
    # Cargar el video
    cap = cv2.VideoCapture(video_path)

    # Verificar si el video se ha abierto correctamente
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return None

    # Leer el primer frame
    ret, frame = cap.read()

    # Verificar si el frame se ha leído correctamente
    if not ret:
        print(f"Error al leer el primer frame del video: {video_path}")
        return None

    # Liberar el objeto VideoCapture
    cap.release()

    print(f"\n Primer frame del video {video_path} extraído exitosamente")

    return frame

def recortar_y_guardar_imagenes(orig_img, result, output_folder):
    # Aumentar el tamaño de las imágenes recolectadas en 30 píxeles por lado
    margin = 30

    # Nombre del video sin la extensión
    video_name = os.path.splitext(os.path.basename(output_folder))[0]

    # Crear la carpeta para este video si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener las cajas y los IDs de seguimiento
    boxes = result[0].boxes.xyxyn.cpu()

    # Dimensiones de la imagen original
    img_height, img_width = orig_img.shape[:2]

    # Iterar sobre las cajas
    for j, box in enumerate(boxes):
        # Desnormalizar las coordenadas de la caja
        left, top, right, bottom = box
        left = int(left * img_width)
        top = int(top * img_height)
        right = int(right * img_width)
        bottom = int(bottom * img_height)

        # Recortar la región de interés (ROI) de la imagen original
        roi = orig_img[max(top - margin, 0):min(bottom + margin, img_height),
                        max(left - margin, 0):min(right + margin, img_width)]

        # Guardar la región de interés como una imagen separada
        cv2.imwrite(os.path.join(output_folder, f"objeto_{j}.jpg"), roi)

    print(f"Imágenes recortadas del video {video_name} guardadas exitosamente")

def procesar_videos(carpetas_videos, modelo, output_folder):
    for carpeta in carpetas_videos:
        # Obtener la lista de archivos en la carpeta de video
        archivos_video = [archivo for archivo in os.listdir(carpeta) if archivo.endswith(".mp4") or archivo.endswith(".avi")]

        for archivo in archivos_video:
            video_path = os.path.join(carpeta, archivo)
            # Extraer el primer frame del video
            primer_frame = extraer_primer_frame_video(video_path)

            if primer_frame is not None:
                # Pasar el primer frame por el modelo
                results = modelo(primer_frame)
                # Crear la carpeta de salida para este video
                video_output_folder = os.path.join(output_folder, os.path.splitext(os.path.basename(video_path))[0])
                # Recortar y guardar las imágenes
                recortar_y_guardar_imagenes(primer_frame, results, video_output_folder)


In [7]:
# Lista de carpetas que contienen videos
carpetas_videos = ["Videos/"]

# Carpeta donde se guardarán las imágenes recortadas
carpeta_objetos_detectados = "objetos_detectados"

# Llamar a la función para procesar los videos
procesar_videos(carpetas_videos, model, carpeta_objetos_detectados)


 Primer frame del video Videos/acA1920-155um__22949301__20210628_094625267.avi extraído exitosamente

0: 1216x1920 8 Movils, 4841.5ms
Speed: 47.5ms preprocess, 4841.5ms inference, 1686.5ms postprocess per image at shape (1, 3, 1216, 1920)
Imágenes recortadas del video acA1920-155um__22949301__20210628_094625267 guardadas exitosamente

 Primer frame del video Videos/acA1920-155um__22949301__20220526_124513538.avi extraído exitosamente

0: 1216x1920 36 Movils, 3407.0ms
Speed: 23.5ms preprocess, 3407.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1216, 1920)
Imágenes recortadas del video acA1920-155um__22949301__20220526_124513538 guardadas exitosamente

 Primer frame del video Videos/acA1920-155um__22949301__20210628_114616840.avi extraído exitosamente

0: 1216x1920 13 Movils, 3263.1ms
Speed: 25.5ms preprocess, 3263.1ms inference, 1.9ms postprocess per image at shape (1, 3, 1216, 1920)
Imágenes recortadas del video acA1920-155um__22949301__20210628_114616840 guardadas exitos

In [8]:
import shutil
archivo = shutil.make_archive('objetos_detectados', 'zip','/content/objetos_detectados')